In [1]:
import pandas as pd #this is for tabular data munging
import numpy as np#
import glob # file listings

# 1) Read Data into dataframe
filename = "bws_h_1e-3_N1e5_L63.out"
rawdata = pd.read_csv(filename, sep = "\t", comment = '#', dtype={'M4': float, 'M5': float, 'M6': float, 'M7': float, 'M8': float})

# 2) Filter moments out
moments = rawdata[["L", "t", "chunk", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8"]]

# 3) Compute mean, standard error of the mean for each moment vs time, and save into a file
time=(moments[(moments['chunk']==1) & (moments['L']==63)]).ix[:,"t"] #make sure this value of L is in the source file***
for L in moments["L"].unique():
    result = []
    result.append(time)
    for M in ("M1","M2","M3","M4","M5","M6", "M7", "M8"):
        print M, L
        alldfs = []
        for i in range(1,1000): # Not 1000, but maximum of chunk number
            dfi = (moments[(moments['chunk']==i) & (moments['L']==L)]).reset_index().ix[:,M]
            alldfs.append(dfi) #append time series to alldfs
        ndf = pd.concat(alldfs,axis=1,ignore_index=True)
        
        [rows,N] = ndf.shape
        dfmean = np.mean(ndf,axis=1) #compute the columns of mean values
        dfstd = np.std(ndf,axis=1) #compute the column of variances
        error = dfstd/np.sqrt(N) #compute the column of errors
        result.append(dfmean) #copy the column of means
        result.append(error) #copy the column of errors
    df = pd.DataFrame(result).transpose()
    df['#L'] = L
    df.columns=['time','1mom','1mom_err','2mom','2mom_err','3mom','3mom_err','4mom','4mom_err','5mom','5mom_err','6mom','6mom_err','7mom','7mom_err','8mom','8mom_err', '#L']
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]  #    OR    df = df.ix[:, cols]
    df.to_csv('./2D_L'+str(L)+'_visited_sites_moments_errors_'+filename,sep='\t',index=False) #save in file#result

# 4) extract last line: mean, standard error of the mean for each moment vs L, and save into a file
alldfs = []
for f in glob.glob('./*_visited_sites_moments_errors'+filename): #iterate through all the files containing *time_moments*
    df = pd.read_csv(f, sep="\t") #read the file
    [rows,cols]=df.shape
    dfi = df[df['time']==100000000].reset_index() #extract last row
    alldfs.append(dfi) #append time series to alldfs

ndf = pd.concat(alldfs)
ndf=ndf.drop('index',1)
ndf=ndf.drop('time',1)
ndf=ndf.sort(['#L'])
ndf.to_csv('./2D_totalvisitedsites_moments_errors_vs_L'+filename,sep='\t',index=False) #save in file#result

KeyError: "['L' 't' 'chunk' 'M1' 'M2' 'M3' 'M4' 'M5' 'M6' 'M7' 'M8'] not in index"

In [ ]:
dfi

In [17]:
ndf

,#L,time,1mom,1mom_err,2mom,2mom_err,3mom,3mom_err,4mom,4mom_err,5mom,5mom_err,6mom,6mom_err,7mom,7mom_err,8mom,8mom_err
0,63,100000000.0,4940.0,0.0,745330.0,0.0,2.844390e+08,0.0,1.224884e+11,0.0,5.438482e+13,0.0,2.441541e+16,0.0,1.101979e+19,0.0,4.990253e+21,0.0
0,127,100000000.0,8420.0,0.0,17423598.0,0.0,6.825056e+10,0.0,2.766279e+14,0.0,1.126677e+18,0.0,4.592634e+21,0.0,1.872355e+25,0.0,7.633545e+28,0.0
